# Visualising Eye Movement

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import numpy as np

import matplotlib
matplotlib.use("Agg") 

In [2]:
import sys
import os

sys.path.append(os.path.abspath("../src"))  # Add folder_a to sys.path
from config import *

In [84]:
os.path.abspath("../src")

'/mnt/c/Users/idwe/Documents/Github/Analyzing-concussions-through-eyetracking-measurements/src'

In [3]:
experiment = "FITTS_LAW"
participant_ids = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_samples.pq", 
        columns=["participant_id"]
    )

participant_ids = participant_ids["participant_id"].unique()

In [4]:
# Read data for participant
participant_id = participant_ids[0]
filters = [('participant_id', '=', str(participant_id))]

samples_df = pd.read_parquet(
    f"{PREPROCESSED_DIR}/{experiment}_samples.pq",
    filters=[('participant_id', '=', participant_id)]
    )

events_df = pd.read_parquet(
    f"{PREPROCESSED_DIR}/{experiment}_events.pq",
    filters=[('participant_id', '=', participant_id)]
    )

In [5]:
events_df.head()

,experiment,participant_id,trial_id,time,event,eye,distance,target_width,colour,stimulus_x,...,x,y,avg_pupil_size,start_x,start_y,end_x,end_y,amplitude,peak_velocity,stimulus_active
0,FITTS_LAW,106,0,0,START,<NA>,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,FITTS_LAW,106,0,7,EFIX,L,NaN,NaN,<NA>,NaN,...,788.1,501.3,1053.0,NaN,NaN,NaN,NaN,NaN,NaN,False
2,FITTS_LAW,106,0,7,EFIX,R,NaN,NaN,<NA>,NaN,...,735.9,542.0,1019.0,NaN,NaN,NaN,NaN,NaN,NaN,False
3,FITTS_LAW,106,0,33,TRIAL_VAR_DATA,<NA>,7.48,0.75,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,FITTS_LAW,106,0,33,FIXPOINT,<NA>,7.48,0.75,255 0 0,794.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [11]:
trial_id=1

sample_df = samples_df.loc[samples_df["trial_id"]==trial_id,:]

# Extract fixpoints
event_df = events_df.loc[events_df["trial_id"]==trial_id,:]
fixpoints_df = event_df[event_df["event"]=="FIXPOINT"].loc[:,["participant_id", "trial_id", "time", "event", "colour", "stimulus_x", "stimulus_y"]]

# Insert fixpoints in sample data
# Ensure you are modifying actual copies
sample_df = sample_df.copy()
fixpoints_df = fixpoints_df.copy()

# Sort by participant_id, trial_id, then time (important for merge_asof with 'by')
sample_df = sample_df.sort_values(["time"])
fixpoints_df = fixpoints_df.sort_values(["time"])

# Rename 'colour' column to 'fixpoint' so it's ready to merge
fixpoints_df = fixpoints_df.rename(columns={"colour": "fixpoint"})

# Perform a backward-looking join: for each row in sample_df, find the most recent fixpoint time
sample_df = pd.merge_asof(
    sample_df,
    fixpoints_df,
    on=["time"],
    by=["participant_id", "trial_id"],
    direction="backward"
    # tolerance=10
)

sample_df["fixpoint"] = sample_df["fixpoint"].map({RED:"red", GREEN:"green", BLUE:"blue", WHITE:"white"})

In [12]:
x = sample_df.copy()
x_sampled = x[x.index % 100 == 0]

# Apply Seaborn style
sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

# Initialize the scatter plot object
scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_fixpoint = ax.scatter([], [], label="Fixpoint", alpha=0.6, s=50)  # 's' is for size, 'c' is for color

x_min, x_max = 0, 1920 
y_min, y_max = 0, 1080 
ax.set_xlim([x_min, x_max]) 
ax.set_ylim([y_min, y_max]) 

# Add legend to distinguish between the three points
ax.legend()

def animate(i):
    # Update the scatter plot data for each frame
    
    # Update coordinates
    coords_right = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_right"].iloc[i]]
    coords_left = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_left"].iloc[i]] 
    coords_fixpoint = np.c_[x_sampled["stimulus_x"].iloc[i], x_sampled["stimulus_y"].iloc[i]] 

    scatter_right.set_offsets(coords_right)    
    scatter_left.set_offsets(coords_left)
    scatter_fixpoint.set_offsets(coords_fixpoint)

    # Update colour based on fixpoint
    fixpoint_color = x_sampled["fixpoint"].iloc[i]
    if pd.isna(fixpoint_color):
        fixpoint_color = "gray"
    elif fixpoint_color == "white":
        fixpoint_color = "black"
    scatter_fixpoint.set_color([fixpoint_color])

    return scatter_right, scatter_left#, scatter_fixpoint

anim = animation.FuncAnimation(
    fig, 
    animate, 
    frames = len(x_sampled), 
    interval=100,
    blit = False
)

anim.save(f"Animations/{experiment}_{participant_id}_{trial_id}.mp4", writer="ffmpeg", fps=30)


/tmp/ipykernel_17085/1757578002.py:10: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
/tmp/ipykernel_17085/1757578002.py:11: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
2025-04-28 10:26:21,168 - INFO - animation.save:1076 - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2025-04-28 10:26:21,171 - INFO - animation._run:319 - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 600x600 -pix_fmt rgba -framerate 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y Animations/FITTS_LAW_106_1.mp4
